In [1]:
import os
import pandas as pd
import json
from IPython.display import display, Markdown, Latex

from common.utils import RESULTS_DIR, EVAL_SIZE, filename_to_obj, remove_index

In [2]:
def results_as_pandas(filename):
    path = os.path.join(RESULTS_DIR, filename)
    with open(path, "r") as f:
        data = f.readlines()
    data = [json.loads(d) for d in data]
    data = pd.DataFrame(data)

    params = filename_to_obj(filename)
    for k, v in params.items():
        data[k] = v

    data = data.explode("evaluations")
    data = data.rename_axis("question_idx").reset_index()

    data = pd.concat([data, data["evaluations"].apply(pd.Series)], axis=1)
    evaluation_keys = data["evaluations"].apply(pd.Series).columns
    for col in evaluation_keys:
        data = pd.concat([data, data[col].apply(pd.Series).add_prefix(f"{col}/")], axis=1)
        data = data.drop(columns=col)
    data = data.drop(columns=["evaluations"])

    return data


files = os.listdir(RESULTS_DIR)
params_names = list(filename_to_obj(files[0]).keys())
all_results = pd.concat([results_as_pandas(f) for f in files])
all_results.head()

,question_idx,question_id,llm,prompt_id,temperature,nli,nlg,citations/ais_recall,citations/ais_precision,citations/n_sentences,...,citations/n_overcitations,citations/sentences,citations/supported,citations/citations,citations/correct_citations,citations/out_of_range,correctness/answer_overlap,correctness/answer_entail,correctness/citations_recall,quality/answer_relevance
0,0,5abab42e55429955dce3eed2,qwen1_5-14b-chat-q8_0,1,0.1,t5_xxl_true_nli_mixture,mistral-7b-instruct-v0.2.Q8_0,0.000000,0.5,3,...,0,[Kevin Crowley is the professional lacrosse pl...,"[0, 0, 0]","[[], [], [3, 4]]","[[], [], [True]]","[0, 0, 0]",1.0,0.0,1.0,0.546523
1,0,5abab42e55429955dce3eed2,qwen1_5-14b-chat-q8_0,1,0.1,t5_xxl_true_nli_mixture,mistral-7b-instruct-v0.2.Q8_0,0.000000,1.0,2,...,0,[Kevin Crowley is the professional lacrosse pl...,"[0, 0]","[[], [4]]","[[], [True]]","[0, 0]",1.0,0.0,0.5,0.650574
2,0,5abab42e55429955dce3eed2,qwen1_5-14b-chat-q8_0,1,0.1,t5_xxl_true_nli_mixture,mistral-7b-instruct-v0.2.Q8_0,0.333333,1.0,3,...,0,[Kevin Crowley is the professional lacrosse pl...,"[0, 0, 1]","[[], [], [2]]","[[], [], [True]]","[0, 0, 0]",1.0,0.0,0.0,0.480813
3,1,5a761900554299109176e648,qwen1_5-14b-chat-q8_0,1,0.1,t5_xxl_true_nli_mixture,mistral-7b-instruct-v0.2.Q8_0,1.000000,1.0,1,...,0,"[Flynn Intel Group, a lobbying group House Ide...",[1],"[[0, 1]]","[[True, True]]",[0],1.0,1.0,1.0,0.378449
4,1,5a761900554299109176e648,qwen1_5-14b-chat-q8_0,1,0.1,t5_xxl_true_nli_mixture,mistral-7b-instruct-v0.2.Q8_0,1.000000,1.0,1,...,0,"[Flynn Intel Group, a lobbying group House Ide...",[1],"[[0, 1]]","[[True, True]]",[0],1.0,1.0,1.0,0.378826


In [3]:
all_obj_cols = all_results.select_dtypes(include=["object"]).columns
drop_obj_cols = list(set(all_obj_cols) - set(params_names))
drop_obj_cols.remove("question_id")
print(f"Dropping columns: {drop_obj_cols}")
all_num_results = all_results.drop(columns=drop_obj_cols)

Dropping columns: ['citations/citations', 'citations/correct_citations', 'citations/supported', 'citations/out_of_range', 'citations/sentences']


In [4]:
eval_split = all_num_results[all_num_results["question_idx"] < EVAL_SIZE]
train_split = all_num_results[all_num_results["question_idx"] >= EVAL_SIZE]

In [5]:
def aggregate(split):
    split = split.drop(columns=["question_idx"])
    results_with_std_for_each_question = split.groupby([*params_names, "question_id"]).agg(["mean", "std"])
    results_for_each_model = results_with_std_for_each_question.groupby(params_names)
    results = results_for_each_model.mean()
    results["n_questions"] = results_for_each_model.size()
    return results

eval_results = aggregate(eval_split)
train_results = aggregate(train_split)

if eval_results["n_questions"].nunique() != 1:
    print("Warning: not all rows in evaluation have the same number of examples")

In [6]:
display(Markdown("### Prompts comparison"))
eval_results[eval_results.index.get_level_values("llm") == "mistral-7b-instruct-v0.2.Q8_0"]

### Prompts comparison

citations/ais_recall  \
                                                                                                                          mean   
llm                           prompt_id temperature nli                     nlg                                                  
mistral-7b-instruct-v0.2.Q8_0 1         0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0             0.549818   
                              2         0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0             0.488163   
                              3         0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0             0.512314   

                                                                                                                     \
                                                                                                                std   
llm                           prompt_id temperature nli                     nlg                                       
mistral-7b-instruct-v0.2.Q8_0 1         0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0  0.056448   
                              2         0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0  0.075649   
                              3         0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0  0.055778   

                                                                                                          citations/ais_precision  \
                                                                                                                             mean   
llm                           prompt_id temperature nli                     nlg                                                     
mistral-7b-instruct-v0.2.Q8_0 1         0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0                0.876011   
                              2         0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0                0.843323   
                              3         0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0                0.877921   

                                                                                                                     \
                                                                                                                std   
llm                           prompt_id temperature nli                     nlg                                       
mistral-7b-instruct-v0.2.Q8_0 1         0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0  0.038954   
                              2         0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0  0.054662   
                              3         0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0  0.038959   

                                                                                                          citations/n_sentences  \
                                                                                                                           mean   
llm                           prompt_id temperature nli                     nlg                                                   
mistral-7b-instruct-v0.2.Q8_0 1         0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0              2.853333   
                              2         0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0              3.190000   
                              3         0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0              2.420000   

                                                                                                                     \
                                                                                                                std   
llm                           prompt_id temperature nli                

In [7]:
display(Markdown("### Evaluation results"))
eval_display = eval_results[eval_results.index.get_level_values("prompt_id") == "1"]
eval_display = remove_index(eval_display, "prompt_id")
eval_display

### Evaluation results

prompt_id  \
                                                                                                              
llm                             temperature nli                     nlg                                       
gpt-3.5-turbo-0125              0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0         1   
mistral-7b-instruct-v0.2.Q8_0   0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0         1   
mixtral-8x7b-instruct-v0.1.Q8_0 0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0         1   
qwen1_5-14b-chat-q8_0           0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0         1   
qwen1_5-7b-chat-q8_0            0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0         1   

                                                                                                  citations/ais_recall  \
                                                                                                                  mean   
llm                             temperature nli                     nlg                                                  
gpt-3.5-turbo-0125              0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0             0.729222   
mistral-7b-instruct-v0.2.Q8_0   0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0             0.549818   
mixtral-8x7b-instruct-v0.1.Q8_0 0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0             0.603619   
qwen1_5-14b-chat-q8_0           0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0             0.628333   
qwen1_5-7b-chat-q8_0            0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0             0.231778   

                                                                                                             \
                                                                                                        std   
llm                             temperature nli                     nlg                                       
gpt-3.5-turbo-0125              0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0  0.075490   
mistral-7b-instruct-v0.2.Q8_0   0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0  0.056448   
mixtral-8x7b-instruct-v0.1.Q8_0 0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0  0.082348   
qwen1_5-14b-chat-q8_0           0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0  0.042339   
qwen1_5-7b-chat-q8_0            0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0  0.069444   

                                                                                                  citations/ais_precision  \
                                                                                                                     mean   
llm                             temperature nli                     nlg                                                     
gpt-3.5-turbo-0125              0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0                0.991389   
mistral-7b-instruct-v0.2.Q8_0   0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0                0.876011   
mixtral-8x7b-instruct-v0.1.Q8_0 0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0                0.922921   
qwen1_5-14b-chat-q8_0           0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0                0.978333   
qwen1_5-7b-chat-q8_0            0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0                0.863667   

                                                                                                             \
                                                                                                        std   
llm                             temperature nli                     nlg                                       
gpt-3.5-t

In [8]:
display(Markdown("### Training results"))
train_display = train_results[train_results.index.get_level_values("prompt_id") == "1"]
train_display = remove_index(train_display, "prompt_id")
train_display

### Training results

prompt_id  \
                                                                                                              
llm                             temperature nli                     nlg                                       
mixtral-8x7b-instruct-v0.1.Q8_0 0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0         1   

                                                                                                  citations/ais_recall  \
                                                                                                                  mean   
llm                             temperature nli                     nlg                                                  
mixtral-8x7b-instruct-v0.1.Q8_0 0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0             0.624332   

                                                                                                             \
                                                                                                        std   
llm                             temperature nli                     nlg                                       
mixtral-8x7b-instruct-v0.1.Q8_0 0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0  0.085718   

                                                                                                  citations/ais_precision  \
                                                                                                                     mean   
llm                             temperature nli                     nlg                                                     
mixtral-8x7b-instruct-v0.1.Q8_0 0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0                0.901555   

                                                                                                            \
                                                                                                       std   
llm                             temperature nli                     nlg                                      
mixtral-8x7b-instruct-v0.1.Q8_0 0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0  0.04351   

                                                                                                  citations/n_sentences  \
                                                                                                                   mean   
llm                             temperature nli                     nlg                                                   
mixtral-8x7b-instruct-v0.1.Q8_0 0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0              2.661476   

                                                                                                             \
                                                                                                        std   
llm                             temperature nli                     nlg                                       
mixtral-8x7b-instruct-v0.1.Q8_0 0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0  0.336632   

                                                                                                  citations/n_total_citations  \
                                                                                                                         mean   
llm                             temperature nli                     nlg                                                         
mixtral-8x7b-instruct-v0.1.Q8_0 0.1         t5_xxl_true_nli_mixture mistral-7b-instruct-v0.2.Q8_0                    3.073878   

                                                                                                             \
                                                                                                        std   
llm                             temperature nli                     nlg                             